In [62]:
# imports
import pandas as pd
import matplotlib.pyplot

In [101]:
# read files 
private_data_original = pd.read_csv('../data/private_dataB.csv')
public_data_register = pd.read_csv('../data/public_data_registerB.csv')
public_data_results = pd.read_csv('../data/public_data_resultsB.csv')

In [102]:
# read anony
anonymized_data = pd.read_csv('../kanon2.csv')
anonymized_data = anonymized_data[['party', 'sex', 'zip', 'age', 'citizenship', 'education', 'evote']]

In [103]:
anonymized_data.head()

,party,sex,zip,age,citizenship,education,evote
0,Red,Female,2300.0,4.0,Denmark,Vocational bachelors educations,0
1,Red,Female,2200.0,4.0,Denmark,Vocational Education and Training (VET),0
2,Green,Female,2100.0,2.0,Denmark,Vocational Education and Training (VET),1
3,Green,Male,2400.0,3.0,Denmark,Vocational Education and Training (VET),0
4,Green,Male,2200.0,1.0,Denmark,Masters programmes,0


In [104]:
# just checking for making sure
anonymized_data[anonymized_data['zip'].isna()]

,party,sex,zip,age,citizenship,education,evote
81,Green,Female,NaN,4.0,Other,Primary education,0
120,Green,Male,NaN,2.0,Other,Masters programmes,0
183,Red,Female,NaN,4.0,Other,Vocational bachelors educations,0
192,Red,Female,NaN,4.0,Other,Primary education,0


In [105]:
anonymized_data.describe()

,zip,age,evote
count,196.000000,179.000000,200.000000
mean,2266.326531,2.614525,0.260000
std,106.162795,0.931358,0.439735
min,2100.000000,1.000000,0.000000
25%,2200.000000,2.000000,0.000000
50%,2300.000000,3.000000,0.000000
75%,2400.000000,3.000000,1.000000
max,2400.000000,4.000000,1.000000


In [106]:
anonymized_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   party        200 non-null    object 
 1   sex          200 non-null    object 
 2   zip          196 non-null    float64
 3   age          179 non-null    float64
 4   citizenship  199 non-null    object 
 5   education    200 non-null    object 
 6   evote        200 non-null    int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 11.1+ KB


In [107]:
anonymized_data['age'].value_counts()

3.0    73
2.0    50
4.0    31
1.0    25
Name: age, dtype: int64

## Creating the auxiliary data

In [143]:
no_dk_df = private_data_original[private_data_original['citizenship'] != "Denmark"]
no_dk_df['public_age'] = no_dk_df['dob'].apply(lambda x: 2020 - int(x[:4]))

In [145]:
no_dk_df.head()

,Unnamed: 0,name,sex,evote,dob,zip,education,citizenship,marital_status,party,public_age
29,29,"Carlos, Caleb",Male,0,1961-10-25,2400,Vocational Education and Training (VET),Italy,Married/separated,Green,59
30,30,"Walker, Remington",Male,0,1954-08-27,2300,Masters programmes,Poland,Married/separated,Red,66
70,70,"Chang, Phurba",Male,0,1987-05-11,2100,Primary education,Brazil,Never married,Green,33
73,73,"Rae, Phillip",Male,0,1971-05-19,2400,Vocational Education and Training (VET),Turkey,Never married,Green,49
81,81,"Newson, Jasmine",Female,0,1950-02-09,2300,Primary education,Pakistan,Never married,Green,70


In [147]:
no_dk_df[['public_age', 'citizenship']].to_csv('auxiliary0.csv', index = False, header = True)

## Doing transforms you know they did too

In [110]:
public_data_register.head()

,Unnamed: 0,name,sex,dob,zip,citizenship,marital_status,last_voted
0,0,"Le, Ian",Male,1998-08-30,2300,Denmark,Never married,1
1,1,"Wang, Caleb",Male,1996-04-23,2200,Denmark,Never married,1
2,2,"Carranco, Raquel",Female,1994-08-06,2200,Denmark,Never married,1
3,3,"Williams, Geona",Female,1995-04-22,2100,Sweden,Never married,2
4,4,"Gonzales, Edward",Male,2000-05-13,2100,Denmark,Never married,1


In [111]:
# age calc the same way
public_data_register['public_age'] = public_data_register['dob'].apply(lambda x: 2020 - int(x[:4]))

In [112]:
public_data_register.head()

,Unnamed: 0,name,sex,dob,zip,citizenship,marital_status,last_voted,public_age
0,0,"Le, Ian",Male,1998-08-30,2300,Denmark,Never married,1,22
1,1,"Wang, Caleb",Male,1996-04-23,2200,Denmark,Never married,1,24
2,2,"Carranco, Raquel",Female,1994-08-06,2200,Denmark,Never married,1,26
3,3,"Williams, Geona",Female,1995-04-22,2100,Sweden,Never married,2,25
4,4,"Gonzales, Edward",Male,2000-05-13,2100,Denmark,Never married,1,20


In [113]:
citizenship_transformer = lambda x: "Denmark" if x == "Denmark" else "Other"
age_transformer = lambda x: 1 if x <= 30 else (2 if x <= 45 else (3 if x <= 65 else 4))

In [114]:
public_data_register['encoded_age'] = public_data_register['public_age'].apply(age_transformer)
public_data_register['encoded_citizenship'] = public_data_register['citizenship'].apply(citizenship_transformer)

In [115]:
# filter out anyone who has not casted a vote
public_data_register = public_data_register[public_data_register['last_voted'] != 2]

In [116]:
merged_df = public_data_register.merge(anonymized_data, on = 'sex', how = 'outer')

In [117]:
print(merged_df.shape)
merged_df.head()

(106214, 17)


,Unnamed: 0,name,sex,dob,zip_x,citizenship_x,marital_status,last_voted,public_age,encoded_age,encoded_citizenship,party,zip_y,age,citizenship_y,education,evote
0,0,"Le, Ian",Male,1998-08-30,2300,Denmark,Never married,1,22,1,Denmark,Green,2400.0,3.0,Denmark,Vocational Education and Training (VET),0
1,0,"Le, Ian",Male,1998-08-30,2300,Denmark,Never married,1,22,1,Denmark,Green,2200.0,1.0,Denmark,Masters programmes,0
2,0,"Le, Ian",Male,1998-08-30,2300,Denmark,Never married,1,22,1,Denmark,Red,2300.0,2.0,Denmark,Vocational bachelors educations,0
3,0,"Le, Ian",Male,1998-08-30,2300,Denmark,Never married,1,22,1,Denmark,Red,2300.0,3.0,Denmark,Vocational Education and Training (VET),1
4,0,"Le, Ian",Male,1998-08-30,2300,Denmark,Never married,1,22,1,Denmark,Red,2400.0,3.0,Denmark,Masters programmes,0


In [118]:
merged_df['name'].value_counts().mean()

100.2965061378659

In [119]:
merged_df['party'].value_counts()

Green           66634
Red             37458
Invalid vote     2122
Name: party, dtype: int64

In [120]:
# compare filterer function
def citizenship_compare(x):
    return x[14] == x[10]

def pd_or(x):
    return x[17] or x[18]

In [121]:
# citizenship cleaning

merged_df['citizenship_na'] = merged_df['citizenship_y'].isna()
merged_df['citizenship_match'] = merged_df.apply(citizenship_compare, axis = 1)
merged_df['citizenship_stays'] = merged_df.apply(pd_or, axis = 1)

merged_df = merged_df[merged_df['citizenship_stays']]
del merged_df['citizenship_na']
del merged_df['citizenship_match']
del merged_df['citizenship_stays']

print(merged_df['name'].value_counts().mean())
merged_df.head()

85.45420207743155


,Unnamed: 0,name,sex,dob,zip_x,citizenship_x,marital_status,last_voted,public_age,encoded_age,encoded_citizenship,party,zip_y,age,citizenship_y,education,evote
0,0,"Le, Ian",Male,1998-08-30,2300,Denmark,Never married,1,22,1,Denmark,Green,2400.0,3.0,Denmark,Vocational Education and Training (VET),0
1,0,"Le, Ian",Male,1998-08-30,2300,Denmark,Never married,1,22,1,Denmark,Green,2200.0,1.0,Denmark,Masters programmes,0
2,0,"Le, Ian",Male,1998-08-30,2300,Denmark,Never married,1,22,1,Denmark,Red,2300.0,2.0,Denmark,Vocational bachelors educations,0
3,0,"Le, Ian",Male,1998-08-30,2300,Denmark,Never married,1,22,1,Denmark,Red,2300.0,3.0,Denmark,Vocational Education and Training (VET),1
4,0,"Le, Ian",Male,1998-08-30,2300,Denmark,Never married,1,22,1,Denmark,Red,2400.0,3.0,Denmark,Masters programmes,0


In [122]:
merged_df['party'].value_counts()

Green           56363
Red             32161
Invalid vote     1972
Name: party, dtype: int64

In [123]:
# zip filtering
def zip_compare(x):
    return x[4] == x[12]

In [124]:
# zip cleaning

merged_df['zip_na'] = merged_df['zip_y'].isna()
merged_df['zip_match'] = merged_df.apply(zip_compare, axis = 1)
merged_df['zip_stays'] = merged_df.apply(pd_or, axis = 1)

merged_df = merged_df[merged_df['zip_stays']]
del merged_df['zip_na']
del merged_df['zip_match']
del merged_df['zip_stays']

print(merged_df['name'].value_counts().mean())
merged_df.head()

22.299338999055713


,Unnamed: 0,name,sex,dob,zip_x,citizenship_x,marital_status,last_voted,public_age,encoded_age,encoded_citizenship,party,zip_y,age,citizenship_y,education,evote
2,0,"Le, Ian",Male,1998-08-30,2300,Denmark,Never married,1,22,1,Denmark,Red,2300.0,2.0,Denmark,Vocational bachelors educations,0
3,0,"Le, Ian",Male,1998-08-30,2300,Denmark,Never married,1,22,1,Denmark,Red,2300.0,3.0,Denmark,Vocational Education and Training (VET),1
6,0,"Le, Ian",Male,1998-08-30,2300,Denmark,Never married,1,22,1,Denmark,Green,2300.0,2.0,Denmark,Vocational Education and Training (VET),0
16,0,"Le, Ian",Male,1998-08-30,2300,Denmark,Never married,1,22,1,Denmark,Green,2300.0,2.0,Denmark,Masters programmes,1
18,0,"Le, Ian",Male,1998-08-30,2300,Denmark,Never married,1,22,1,Denmark,Red,2300.0,4.0,Denmark,Vocational Education and Training (VET),0


In [125]:
merged_df['party'].value_counts()

Green           14308
Red              8731
Invalid vote      576
Name: party, dtype: int64

In [126]:
# age filtering
def age_compare(x):
    return x[13] == x[9]

def test(x):
    return x[13]

In [127]:
# age cleaning

merged_df['age_na'] = merged_df['age'].isna()
merged_df['age_match'] = merged_df.apply(age_compare, axis = 1)
merged_df['age_stays'] = merged_df.apply(pd_or, axis = 1)

merged_df = merged_df[merged_df['age_stays']]
del merged_df['age_na']
del merged_df['age_match']
del merged_df['age_stays']

print(merged_df['name'].value_counts().mean())
merged_df.head()

8.500478468899521


,Unnamed: 0,name,sex,dob,zip_x,citizenship_x,marital_status,last_voted,public_age,encoded_age,encoded_citizenship,party,zip_y,age,citizenship_y,education,evote
36,0,"Le, Ian",Male,1998-08-30,2300,Denmark,Never married,1,22,1,Denmark,Green,2300.0,NaN,Denmark,Vocational bachelors educations,1
47,0,"Le, Ian",Male,1998-08-30,2300,Denmark,Never married,1,22,1,Denmark,Green,2300.0,NaN,Denmark,Vocational Education and Training (VET),0
82,0,"Le, Ian",Male,1998-08-30,2300,Denmark,Never married,1,22,1,Denmark,Red,2300.0,NaN,Denmark,Masters programmes,1
95,1,"Wang, Caleb",Male,1996-04-23,2200,Denmark,Never married,1,24,1,Denmark,Green,2200.0,1.0,Denmark,Masters programmes,0
101,1,"Wang, Caleb",Male,1996-04-23,2200,Denmark,Never married,1,24,1,Denmark,Green,2200.0,1.0,Denmark,Vocational Education and Training (VET),1


In [128]:
merged_df['party'].value_counts()

Green           5503
Red             3148
Invalid vote     232
Name: party, dtype: int64

In [129]:
# evote filtering
def vote_compare(x):
    return x[7] == x[16]

In [130]:
# evote cleaning

merged_df['vote_match'] = merged_df.apply(vote_compare, axis = 1)

merged_df = merged_df[merged_df['vote_match']]
del merged_df['vote_match']

print(merged_df['name'].value_counts().mean())
merged_df.head()

5.390147783251232


,Unnamed: 0,name,sex,dob,zip_x,citizenship_x,marital_status,last_voted,public_age,encoded_age,encoded_citizenship,party,zip_y,age,citizenship_y,education,evote
36,0,"Le, Ian",Male,1998-08-30,2300,Denmark,Never married,1,22,1,Denmark,Green,2300.0,NaN,Denmark,Vocational bachelors educations,1
82,0,"Le, Ian",Male,1998-08-30,2300,Denmark,Never married,1,22,1,Denmark,Red,2300.0,NaN,Denmark,Masters programmes,1
101,1,"Wang, Caleb",Male,1996-04-23,2200,Denmark,Never married,1,24,1,Denmark,Green,2200.0,1.0,Denmark,Vocational Education and Training (VET),1
145,1,"Wang, Caleb",Male,1996-04-23,2200,Denmark,Never married,1,24,1,Denmark,Green,2200.0,NaN,Denmark,Short cycle higher education,1
151,1,"Wang, Caleb",Male,1996-04-23,2200,Denmark,Never married,1,24,1,Denmark,Invalid vote,2200.0,1.0,NaN,Vocational Education and Training (VET),1


In [148]:
merged_df['party'].value_counts()

Green           3222
Red             2100
Invalid vote     149
Name: party, dtype: int64

## check what you can match after all these narrowing downs


In [156]:
# checking if users are obvious
merged_df[['name', 'sex', 'evote', 'zip_y', 'citizenship_y', 'age']].groupby(['sex', 'evote', 'zip_y', 'citizenship_y', 'age', ]).nunique()

name
sex    evote zip_y  citizenship_y age      
Female 0     2100.0 Denmark       1.0    19
                                  2.0    21
             2200.0 Denmark       1.0    12
                                  2.0    16
                                  3.0    46
                                  4.0    13
             2300.0 Denmark       1.0    13
                                  2.0    26
                                  3.0    39
                                  4.0    17
             2400.0 Denmark       2.0    21
                                  3.0    54
                                  4.0    27
       1     2100.0 Denmark       1.0    12
                                  2.0     5
                                  3.0     7
             2200.0 Denmark       1.0     7
                                  2.0    15
             2300.0 Denmark       2.0    12
             2400.0 Denmark       2.0    13
                                  3.0    22
Male   0     2100.0 Denmark       3.0    13
                                  4.0     3
             2200.0 Denmark       1.0    14
                                  2.0    18
                                  3.0    44
                                  4.0    15
             2300.0 Denmark       2.0    17
                                  3.0    41
                                  4.0    15
             2400.0 Denmark       2.0    22
                                  3.0    64
                                  4.0    25
                    Other         3.0     7
       1     2100.0 Denmark       1.0    16
                                  2.0    14
             2200.0 Denmark       1.0     8
                                  2.0    16
             2300.0 Denmark       2.0    15
                                  3.0    12
             2400.0 Denmark       3.0    16

In [139]:
# for testing purposes
male_df = merged_df[merged_df['sex'] == 'Male']
male_other_df = male_df[male_df['encoded_citizenship'] == 'Other']
male_other_df.shape

(59, 17)

In [142]:
male_other_df

,Unnamed: 0,name,sex,dob,zip_x,citizenship_x,marital_status,last_voted,public_age,encoded_age,encoded_citizenship,party,zip_y,age,citizenship_y,education,evote
3065,78,"Duong, Heassam",Male,1991-08-04,2200,Afghanistan,Married/separated,1,29,1,Other,Invalid vote,2200.0,1.0,NaN,Vocational Education and Training (VET),1
15274,502,"Solongo, Cameron",Male,1989-06-29,2400,China,Never married,0,31,2,Other,Green,2400.0,NaN,Other,Bachelors programmes,0
15282,502,"Solongo, Cameron",Male,1989-06-29,2400,China,Never married,0,31,2,Other,Green,NaN,2.0,Other,Masters programmes,0
15334,505,"el-Yamin, Shaamil",Male,1998-03-31,2300,China,Never married,0,22,1,Other,Red,2300.0,NaN,Other,Masters programmes,0
15363,505,"el-Yamin, Shaamil",Male,1998-03-31,2300,China,Never married,0,22,1,Other,Green,2300.0,NaN,Other,Masters programmes,0
18551,599,"Spicely, Treyvon",Male,2000-06-06,2100,China,Never married,0,20,1,Other,Green,2100.0,NaN,Other,Primary education,0
18602,599,"Spicely, Treyvon",Male,2000-06-06,2100,China,Never married,0,20,1,Other,Green,2100.0,NaN,Other,Primary education,0
19000,629,"Pierce, Chase",Male,1989-09-01,2300,Poland,Married/separated,0,31,2,Other,Red,2300.0,NaN,Other,Masters programmes,0
19029,629,"Pierce, Chase",Male,1989-09-01,2300,Poland,Married/separated,0,31,2,Other,Green,2300.0,NaN,Other,Masters programmes,0
19042,629,"Pierce, Chase",Male,1989-09-01,2300,Poland,Married/separated,0,31,2,Other,Green,NaN,2.0,Other,Masters programmes,0


In [133]:
private_data_original['party'].value_counts()

Green           126
Red              70
Invalid vote      4
Name: party, dtype: int64

In [ ]:
private_data_original